In [ ]:
# The script is to save 2-theta map, Azm map, pixel distance map, and polarization scale map using GSAS-II scriptable.
# Use python installed with GSAS-II to execute the script.  Best way: use run_savemaps.bat (edit paths inside)
# An image and an *.imctrl file have to be present in the directory containing this script and the bat file.
# C:\Users\17bmuser\AppData\Local\Continuum\gsas2full\python.exe

import os,sys,glob
from PIL import Image

sys.path.insert(0,'C:/Users/jweng/gsas2full/GSASII')	# where GSASII is located

import GSASIIscriptable as G2sc
import GSASIIimage as G2img


def getmaps(tifname,imctrlname):		# fast integration using the same imctrl and mask
    imlst = gpx.add_image(PathWrap(tifname),fmthint="TIF")
    imlst[0].loadControls(PathWrap(imctrlname))	# change control file name
    Data = imlst[0].data['Image Controls']
    TA = G2img.Make2ThetaAzimuthMap(Data,(0,Data['size'][1]),(0,Data['size'][0]))    #2-theta array
    path1 =  os.path.join(pathmaps,imctrlname)
    im = Image.fromarray(TA[0])
    im.save(os.path.splitext(path1)[0] + '_2thetamap.tif')
    im = Image.fromarray(TA[1])
    im.save(os.path.splitext(path1)[0] + '_azmmap.tif')
    im = Image.fromarray(TA[2])
    im.save(os.path.splitext(path1)[0] + '_pixelsampledistmap.tif')
    im = Image.fromarray(TA[3])
    im.save(os.path.splitext(path1)[0] + '_polscalemap.tif')

def updatelist(ext):			# search for tif files in the main directory
    filelist=glob.glob(ext)
    filelist.sort(key=lambda x: os.path.getctime(x), reverse=False) #sort files by creation time in ascending order
    return filelist

In [ ]:
if __name__ == "__main__":

    cwd = os.getcwd()				# get the current path

    PathWrap = lambda fil: os.path.join(cwd,fil)

    gpx = G2sc.G2Project(filename=PathWrap('calibrations.gpx'))

    newdir = 'maps'					# make a subfolder to store integrated images
    path = os.path.join(cwd,newdir)
    if not os.path.exists(path):			
        os.mkdir(path)
    pathmaps = path

    tiflist = updatelist('*.tif')
    imctrllist = updatelist('*.imctrl')

    if not tiflist:
        print("need at least one tif image in the folder!")
    elif not imctrllist:
        print("no imctrl files in the folder!")
    else:
        for file in imctrllist:
            getmaps(tiflist[0],file)